# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import pickle

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///./data/DisasterResponseData.db')
df = pd.read_sql('select * from DisasterResponse', engine)

Let's make some dummies variables from the columns category ;)

In [3]:
dummies = pd.DataFrame(pd.get_dummies(df['related'], prefix='related', prefix_sep='_'))

In [4]:
df.drop(columns='related', axis=1, inplace=True)
dummies.drop(columns='related_2', axis=1, inplace=True)

In [5]:
df = pd.concat([df, dummies], axis=1)
df.head()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,related_0,related_1
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = text.lower()

    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tok = []
    for tok in tokens:
        clean_tok.append(lemmatizer.lemmatize(tok).strip())

    return clean_tok

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [10]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
target_names = Y_test.columns

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
# Calculate the accuracy for each one
for i in range(len(target_names)):
    print('Target category {}: {} '.format(i, target_names[i]))
    print(classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))
    print('Confusion Matrix: \n {} \n\n'.format(confusion_matrix(y_pred[:, i], Y_test.iloc[:, i])))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, y_pred[:, i])))

Target category 0: request 
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.83      0.49      0.61      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Confusion Matrix: 
 [[5334  570]
 [ 109  541]] 


Accuracy 0.8963991455599634


Target category 1: offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Confusion Matrix: 
 [[6521   33]
 [   0    0]] 


Accuracy 0.9949649069270674


Target category 2: aid_related 
              precision    recall  f1-score   support

           0       0.80      0.84      0.82  

Confusion Matrix: 
 [[6530   24]
 [   0    0]] 


Accuracy 0.9963381141287763


Target category 25: aid_centers 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Confusion Matrix: 
 [[6473   81]
 [   0    0]] 


Accuracy 0.98764113518462


Target category 26: other_infrastructure 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6271
           1       0.00      0.00      0.00       283

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554

Confusion Matrix: 
 [[6271  283]
 [   0    0]] 


Accuracy 0.9568202624351541


Target category 27: weather_relat

In [14]:
#Clear view droped the first column multiclass and multilabeled
print(classification_report(Y_test.values, np.array(y_pred), target_names = target_names))

                        precision    recall  f1-score   support

               request       0.83      0.49      0.61      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.69      0.72      2670
          medical_help       0.70      0.08      0.14       535
      medical_products       0.79      0.09      0.16       344
     search_and_rescue       0.67      0.05      0.09       159
              security       0.25      0.01      0.02       116
              military       0.61      0.07      0.13       200
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.41      0.55       418
                  food       0.86      0.55      0.67       745
               shelter       0.82      0.43      0.56       581
              clothing       0.83      0.10      0.18        98
                 money       0.80      0.06      0.11       133
        missing_people       1.00      

In [15]:
print('Accuracy {}'.format(accuracy_score(Y_test.values, y_pred)))

Accuracy 0.24336283185840707


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfid

In [17]:
parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 150],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, cv=3)
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50; total time=10.3min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50; total time=11.0min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50; total time=12.9min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50; total time=11.2min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50; total time=11.4min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time=12.9min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time=14.6min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time=18.9min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time=16.9min
[CV] END clf__estimator__min_samples_split=2, clf__est

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100, 150]},
             verbose=2)

In [18]:
print(cv.best_params_)
print(cv.best_score_)
print(cv.best_estimator_)

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 150}
0.2563325710741388
Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=4,
                                                                        n_estimators=150),
                                       n_jobs=-1))])


In [19]:
#building new model
optimised_model = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
optimised_model.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000234C2F93550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=4,
                                                                        n_estimators=150),
                                       n_jobs=-1))])

In [21]:
y_pred = optimised_model.predict(X_test)
print(classification_report(Y_test.values, np.array(y_pred), target_names = target_names))

                        precision    recall  f1-score   support

               request       0.83      0.50      0.62      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.71      0.73      2670
          medical_help       0.69      0.10      0.17       535
      medical_products       0.78      0.08      0.15       344
     search_and_rescue       0.71      0.03      0.06       159
              security       0.00      0.00      0.00       116
              military       0.64      0.09      0.16       200
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.42      0.57       418
                  food       0.86      0.59      0.70       745
               shelter       0.80      0.39      0.53       581
              clothing       0.73      0.08      0.15        98
                 money       0.78      0.05      0.10       133
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(), n_jobs=-1))
])

parameters_kn = {'tfidf__use_idf':[True, False],
                 'clf__estimator__weights':['uniform','distance'],
                 'clf__estimator__leaf_size':[25,35]
                }
cv2 = GridSearchCV(pipeline, param_grid = parameters_kn, verbose=2, cv=2)

np.random.seed(42)

kn_model = cv2.fit(X_train, Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True; total time= 6.7min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True; total time= 8.3min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True; total time= 8.7min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True; total time= 8.7min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True; total time= 6.3min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=False; total time= 5.6min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=False; total time= 5.4min
[CV] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=False; total time= 5.3min
[CV] END clf__estimator__leaf_size=25, cl

In [24]:
print(cv2.best_params_)
print(cv2.best_score_)

{'clf__estimator__leaf_size': 25, 'clf__estimator__weights': 'distance', 'tfidf__use_idf': False}
0.18726767195902683


### 9. Export your model as a pickle file

In [22]:
with open('models/classifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.